### Imports

In [1]:
# import public packages
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import colors
from matplotlib.patches import Rectangle
import scipy
import mne
import sys


from mne.time_frequency import tfr_morlet
from mne.baseline import rescale
from scipy.signal import spectrogram, hann, butter, filtfilt, hilbert
from scipy import signal, interpolate, stats
from scipy.interpolate import make_interp_spline, BSpline
from io import open
from importlib import reload

# import own functions
from utils import find_folders
import dat_preproc
import fix_annot_onsets
import mat2fif
import baseline_correction

### Add Directories / Load Files

In [2]:
#reload(find_folders)
onedrive = find_folders.get_onedrive_path()
ftg_path = find_folders.get_onedrive_path("FTG")
print(ftg_path)

C:Users\mathiopv\OneDrive - Charité - Universitätsmedizin Berlin\FTG_PROJECT


In [24]:
percept_ID = 'sub029'
fname = 'syncNEW_sub-029_ses-EphysMedOn01_task-RampUpThres125_acq-StimOnR2b_run-01_ieeg.mat'

raw = mne.io.read_raw_fieldtrip(
    os.path.join(
        ftg_path,
        'data',
        'raw_data',
        'raw_mats',
        percept_ID,
        fname
    ),
    info = None
)

Creating RawArray with float64 data, n_channels=9, n_times=122625
    Range : 0 ... 122624 =      0.000 ...   490.496 secs
Ready.


c:\Users\mathiopv\Anaconda3\lib\site-packages\pymatreader\utils.py:230: UserWarning: Complex objects (like classes) are not supported. They are imported on a best effort base but your mileage will vary.
  warn('Complex objects (like classes) are not supported. '
C:\Users\mathiopv\AppData\Local\Temp\ipykernel_17136\352291883.py:4: RuntimeWarning: Importing FieldTrip data without an info dict from the original file. Channel locations, orientations and types will be incorrect. The imported data cannot be used for source analysis, channel interpolation etc.
  raw = mne.io.read_raw_fieldtrip(
C:\Users\mathiopv\AppData\Local\Temp\ipykernel_17136\352291883.py:4: RuntimeWarning: The supplied FieldTrip structure does not have an elec or grad field. No channel locations will extracted and the kind of channel might be inaccurate.
  raw = mne.io.read_raw_fieldtrip(
C:\Users\mathiopv\AppData\Local\Temp\ipykernel_17136\352291883.py:4: RuntimeWarning: Cannot guess the correct type of channel LFP_Stn_

#### Artefact Rejection & FIF Files

In [260]:
#Interactive plot for artefact rejection
%matplotlib qt 

fig = raw.plot(n_channels = 2, highpass = 5, lowpass = 100, 
    filtorder = 5, duration = 20)


Setting up band-pass filter from 5 - 1e+02 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 20 (effective, after forward-backward)
- Cutoffs at 5.00, 100.00 Hz: -6.02, -6.02 dB



In [261]:
#First time making them
interactive_annot = raw.annotations
raw.annotations.save('Sub029_Bilateral_artefactsAnnotations.csv', 
    overwrite = True)

Channels marked as bad:
none


In [4]:
#Importing them later
interactive_annot = pd.read_csv(
    os.path.join(
        ftg_path,
        'data',
        'raw_data',
        'clean_fifs',
        'sub029',
        'Sub029_Bilateral_artefactsAnnotations.csv'
    )
)

#fix timing in imported annotations
reload(fix_annot_onsets)
new_onsets = fix_annot_onsets.fix_annot_onsets(interactive_annot)
print(new_onsets)

[  0.   24.5  27.7  65.5  81.7  94.7 108.4 113.1 134.6 168.5 187.3 209.
 245.2 271.8 294.1 335.1]


In [5]:
my_annot = mne.Annotations(onset=new_onsets,  # in seconds
                           duration=interactive_annot.duration,  # in seconds, too
                           description=interactive_annot.description,
                           orig_time=raw.info['meas_date'])

reload(mat2fif)
%matplotlib qt
new_raw = mat2fif.mat2fif(raw,my_annot)

Omitting 10660 of 87875 (12.13%) samples, retaining 77215 (87.87%) samples.
Creating RawArray with float64 data, n_channels=6, n_times=77215
    Range : 0 ... 77214 =      0.000 ...   308.856 secs
Ready.
Using matplotlib as 2D backend.


t:\Dokumente\PROJECTS\DYSKINESIA_PROJECT\FTG_GithubCode\mat2fif.py:4: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw2 = raw.copy().set_annotations(my_annot)


Channels marked as bad:
none


In [7]:
#Plot TF plot with artefact rejected data
%matplotlib qt
reload(dat_preproc)
x = new_raw.get_data(reject_by_annotation = 'omit',picks=[0,1])
raw = new_raw
win_samp = 250
noverlap = 0.5
window = hann(win_samp, sym=False)
f, t, Sxx = dat_preproc.fft_rawviz(raw, x, win_samp, noverlap)

### Plot Power Spectra in epochs

In [ ]:
%matplotlib inline
stim_onsets = [1, 30, 110, 240]
labels = ['Ipsi_Stim','NoStim','Contra_Stim','Bilateral']
dur = 20

#for l in range(0,4):
    #plt.plot(np.mean(Sxx[1,:,stim_onsets[l]:stim_onsets[l]+dur],1), label = labels[l])
    #plt.xlim(40, 90)
    #plt.ylim(0,0.5)
tt, Pxx = scipy.signal.welch(Sxx[1,1:20], fs = 250, nperseg = 250, noverlap = 0.25)
plt.plot(np.arange(1,127), np.mean(Pxx[0],1), label = labels[l])

plt.legend(labels)

In [6]:
reload(dat_preproc)
x = new_raw.get_data() #getting only RSTN channel

x1 = x[1, 12500:75000] 

dat_subh = dat_preproc.low_highpass_filter(x1, 60, 65) #filtering for 60-65Hz
dat_ngam = dat_preproc.low_highpass_filter(x1, 82, 87) #filtering for 80-90Hz
dat_bet = dat_preproc.low_highpass_filter(x1, 20, 35) #filtering for 23-35Hz

datall = [dat_bet, dat_subh, dat_ngam] 
labels = ['High Beta [20-35Hz]', 'Subharmonic [60-65Hz]','FTG [82-87Hz]']

In [7]:
def window_rms(a, window_size):
  a2 = np.power(a,2)
  window = np.ones(window_size)/float(window_size)
  return np.sqrt(np.convolve(a2, window, 'valid'))

In [35]:
sm_signal_np.shape

(3, 60501)

In [37]:
sm_signal_np = np.empty(shape = (3, 61501))
sm_signal_np[:] = np.nan

fig, axes = plt.subplots(3, 1, figsize=(12, 5))
wintosmooth = 1000

for idx, dat in enumerate(datall):
    hiltr = hilbert(dat)
    amplitude_envelope = np.abs(hiltr)
    zscore_sign = stats.zscore(amplitude_envelope)

    sm_signal = window_rms(zscore_sign, wintosmooth)
    
    axes[idx].plot(sm_signal)
    #plt.plot(np.arange(0,75000), amplitude_envelope, label = labels[idx]) 
    
    axes[idx].axvline(26250, color = 'b', ls='--', lw=2, label = 'Stim On')
    axes[idx].axvline(50250, color = 'g', ls='--', lw=2, label = 'Stim Off')
    axes[idx].set_ylabel(str(labels[idx])+ ' Env.')
    axes[idx].set_xlim([0, sm_signal.shape[0]])

    if idx == 1 or idx == 2:
        axes[idx].set_ylim(0,2)
    
    sm_signal_np[idx,:] = sm_signal

    #axes[idx].set_xticks(ticks = np.arange(0, 80000, 10000), labels = np.arange(0,320,40))
    plt.xlabel('Time [sec]')

    

plt.suptitle('Smoothing Window: 125 samples')
axes[0].legend(loc='upper right')

plt.show()

In [38]:
x2 = x[5, 12500:75000] 
sm_stim = window_rms(x2, wintosmooth)
sm_stim1 = (sm_stim + 1)/3

In [42]:
newlabs = ['High Beta [20-35Hz]','Subharmonic [60-65Hz]','FTG [82-87Hz]']
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
for idx, dat in enumerate(sm_signal_np):
    ax1.plot(sm_signal_np[idx,:], label = newlabs[idx], lw = 2)
ax2.plot(sm_stim1, label = 'Stimulation', color = 'grey', ls='--', lw=3, alpha = 0.4)
ax1.legend()
ax1.set_ylabel('Z-scored Smoothed Analytic Signal')
ax2.set_ylabel('Stimulation Amplitude [mA]')
ax2.set_yticks(np.arange(0.5, 2.5, 0.25))
ax2.set_yticklabels(np.arange(0.25, 2.25, 0.25))
ax1.set_xlim(0, sm_signal_np.shape[0])
ax1.set_xticks(np.arange(0, 70000, 10000))
ax1.set_xticklabels(np.arange(0, 280, 40))
ax1.set_xlabel('Time [sec]')

plt.show()


In [ ]:
# let's explore some frequency bands
iter_freqs = [
    ('Beta', 13, 35),
    ('SubGamma', 60, 65),
    ('NarrGamma', 65, 90)
]

In [ ]:
new_raw.ch_names

In [ ]:
raw = new_raw
events = mne.find_events(raw, stim_channel='STIM_R_125Hz_60us')
print(events)

In [ ]:
# set epoching parameters
event_id, tmin, tmax = 1, -1., 3.
baseline = None


frequency_map = list()

for band, fmin, fmax in iter_freqs:
    # (re)load the data to save memory

    # bandpass filter
    raw.filter(fmin, fmax, n_jobs=None,  # use more jobs to speed up.
               l_trans_bandwidth=1,  # make sure filter params are the same
               h_trans_bandwidth=1, picks = 'LFP_Stn_R_13')  # in each band and skip "auto" option.

    # epoch
    epochs = mne.Epochs(raw, events, event_id, tmin, tmax, baseline=baseline,
                        picks= 'LFP_Stn_R_13',
                        preload=True)
    # remove evoked response

    # get analytic signal (envelope)
    epochs.apply_hilbert(envelope=True, picks = 'LFP_Stn_R_13')
    frequency_map.append(((band, fmin, fmax), epochs.average(picks = 'LFP_Stn_R_13')))
    del epochs
del raw

In [ ]:
np.sum(average.data, axis = 0).shape

In [ ]:
from mne.stats import bootstrap_confidence_interval

# Helper function for plotting spread
def stat_fun(x):
    """Return sum of squares."""
    return np.sum(x ** 2, axis=0)


# Plot
fig, axes = plt.subplots(3, 1, figsize=(10, 7), sharex=True, sharey=True)
colors = plt.colormaps['winter_r'](np.linspace(0, 1, 3))
for ((freq_name, fmin, fmax), average), color, ax in zip(
        frequency_map, colors, axes.ravel()[::-1]):
    times = average.times * 1e3
    gfp1 = np.sum(average.data ** 2, axis=0)
    gfp = mne.baseline.rescale(gfp1, times, baseline=(None, 0))
    ax.plot(times, gfp, label=freq_name, color=color, linewidth=2.5)
    ax.axhline(0, linestyle='--', color='grey', linewidth=2)
    ci_low, ci_up = bootstrap_confidence_interval(average.data, random_state=0,
                                                  stat_fun=stat_fun)
    ci_low = rescale(ci_low, average.times, baseline=(None, 0))
    ci_up = rescale(ci_up, average.times, baseline=(None, 0))
    ax.fill_between(times, gfp + ci_up, gfp - ci_low, color=color, alpha=0.3)
    ax.grid(True)
    ax.set_ylabel('GFP')
    ax.annotate('%s (%d-%dHz)' % (freq_name, fmin, fmax),
                xy=(0.95, 0.8),
                horizontalalignment='right',
                xycoords='axes fraction')
    ax.set_xlim(-1000, 3000)
    ax.set_ylim(-50,50)

axes.ravel()[-1].set_xlabel('Time [ms]')